In [ ]:
import requests

In [ ]:
def busca_informes_cvm(ano, mes):
  url = 'https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/'
  if ano < 2021 :
    url = url + 'HIST/'
    file_name = 'inf_diario_fi_{:02d}.zip'.format(ano)
  else :
    file_name = 'inf_diario_fi_{:02d}{:02d}.zip'.format(ano,mes)
  
  download = requests.get(url+file_name)
  with open(file_name, "wb") as arquivo_cvm:
    arquivo_cvm.write(download.content)
  arquivo_zip = zipfile.ZipFile(file_name)
  dados = pd.read_csv(arquivo_zip.open(arquivo_zip.namelist()[0]), sep = ";", encoding = 'ISO-8859-1')

  import os
  os.remove(file_name)

  return dados

def busca_cadastro_cvm(): 
  url = 'https://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv'
  dados = pd.read_csv(url, sep=';', encoding='ISO-8859-1',
                      usecols=['TP_FUNDO', 'CNPJ_FUNDO', 'DENOM_SOCIAL', 
                               'SIT','CLASSE', 'RENTAB_FUNDO', 'CONDOM', 
                               'TRIB_LPRAZO', 'PUBLICO_ALVO', 'TAXA_PERFM',
                               'INF_TAXA_PERFM', 'TAXA_ADM', 'INF_TAXA_ADM', 
                               'CLASSE_ANBIMA'],low_memory=False)
  dados = dados[dados['SIT']=='EM FUNCIONAMENTO NORMAL']
  return dados

In [ ]:
dados_fundo = busca_informes_cvm(ano=2020, mes=12)
dados_cadastro = busca_cadastro_cvm()

In [ ]:
dados_cadastro = busca_cadastro_cvm()[['CNPJ_FUNDO', 'DENOM_SOCIAL']]
# dados_cadastro = dados_cadastro[['CNPJ_FUNDO', 'DENOM_SOCIAL']]
dados_cadastro = dados_cadastro.drop_duplicates()
dados_cadastro.head(3)

In [ ]:
data_inicio_mes = (dados_fundos['DT_COMPTC'].sort_values(ascending = True).unique())[0]
data_fim_mes = (dados_fundos['DT_COMPTC'].sort_values(ascending = True).unique())[-1]

dados_fundos_filtrado = dados_fundos[(dados_fundos['DT_COMPTC'].isin([data_inicio_mes, data_fim_mes]))]
dados_fundos_filtrado.head(3)

In [ ]:
base_final = pd.merge(dados_fundos_filtrado, dados_cadastro, how = "left",
                      left_on = ["CNPJ_FUNDO"], right_on = ["CNPJ_FUNDO"])

base_final.head(3)

In [ ]:
busca_fundo = base_final[base_final['DENOM_SOCIAL'].str.contains("V8 CASH", na = False)]
busca_fundo[['CNPJ_FUNDO' , 'DT_COMPTC' , 'DENOM_SOCIAL']]

Patrimônio e performance do fundo em dezembro

In [ ]:
cnpj = "30.509.221/0001-50"
busca_fundo[(busca_fundo['CNPJ_FUNDO'] == cnpj)]['DENOM_SOCIAL'].values

In [ ]:
fundo = busca_fundo[(busca_fundo['CNPJ_FUNDO'] == cnpj) &
                    (busca_fundo['DT_COMPTC'] == "2023-12-29")]

patrimonio_do_fundo =  "R$ " + str((fundo['VL_PATRIM_LIQ'].iloc[0]/1000000).round(2)) + "MM"

patrimonio_do_fundo

In [ ]:
fundo_fic = busca_fundo[(busca_fundo['CNPJ_FUNDO'] == cnpj)]

retorno_dez = ("Retorno fundo: " +
            str(((fundo_fic['VL_QUOTA'].iloc[-1]/fundo_fic['VL_QUOTA'].iloc[0] - 1)
                 * 100).round(2))
               + "%")

retorno_dez

In [ ]:
# mostra os melhores que satifazem as restrições
minimo_cotistas = 5000
top = 5
classe = 'multimercado'

cadastro      = cadastro[cadastro['SIT'] == 'EM FUNCIONAMENTO NORMAL']
fundos        = informes[informes['NR_COTST'] >= minimo_cotistas]
print(fundos)
cnpj_informes = fundos['CNPJ_FUNDO'].drop_duplicates()
fundos = fundos.pivot(index='DT_COMPTC', columns='CNPJ_FUNDO')  
cotas_normalizadas = fundos['VL_QUOTA'] / fundos['VL_QUOTA'].iloc[0]
  
if classe == 'multimercado':
  cnpj_cadastro      = cadastro[cadastro['CLASSE'] == 'Fundo Multimercado']['CNPJ_FUNDO']   
  cotas_normalizadas = cotas_normalizadas[cnpj_cadastro[cnpj_cadastro.isin(cnpj_informes)]]

if classe == 'acoes':
  cnpj_cadastro      = cadastro[cadastro['CLASSE'] == 'Fundo de Ações']['CNPJ_FUNDO']   
  cotas_normalizadas = cotas_normalizadas[cnpj_cadastro[cnpj_cadastro.isin(cnpj_informes)]]

if classe == 'rendafixa':
  cnpj_cadastro      = cadastro[cadastro['CLASSE'] == 'Fundo de Renda Fixa']['CNPJ_FUNDO']   
  cotas_normalizadas = cotas_normalizadas[cnpj_cadastro[cnpj_cadastro.isin(cnpj_informes)]]

if classe == 'cambial':
  cnpj_cadastro      = cadastro[cadastro['CLASSE'] == 'Fundo Cambial']['CNPJ_FUNDO']   
  cotas_normalizadas = cotas_normalizadas[cnpj_cadastro[cnpj_cadastro.isin(cnpj_informes)]]

melhores = pd.DataFrame()
melhores['retorno(%)'] = (cotas_normalizadas.iloc[-1].sort_values(ascending=False) - 1) * 100

for cnpj in melhores.index:
  fundo = cadastro[cadastro['CNPJ_FUNDO'] == cnpj]
  melhores.at[cnpj, 'Fundo de Investimento'] = fundo['DENOM_SOCIAL'].values[0]
  melhores.at[cnpj, 'Classe'] = fundo['CLASSE'].values[0]
  melhores.at[cnpj, 'PL'] = fundo['VL_PATRIM_LIQ'].values[0]


pd.set_option("display.max_colwidth", 150)
pd.set_option("display.max_rows", 477)

print(melhores['Fundo de Investimento'][:top])

In [ ]:
# os n piores
n=10
pd.set_option("display.max_rows", 399)
piores = pd.DataFrame()
piores['retorno(%)'] = (cotas_normalizadas.iloc[-1].sort_values(ascending=True)[:n] - 1) * 100

for cnpj in piores.index:
  fundo = cadastro[cadastro['CNPJ_FUNDO'] == cnpj]
  piores.at[cnpj, 'Fundo de Investimento'] = fundo['DENOM_SOCIAL'].values[0]
  piores.at[cnpj, 'Classe'] = fundo['CLASSE'].values[0]
  piores.at[cnpj, 'PL'] = fundo['VL_PATRIM_LIQ'].values[0]

piores['Fundo de Investimento']